In [1]:
# !pip install -U spacy

In [2]:
# !pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.0.0/ru_core_news_sm-3.0.0.tar.gz


In [50]:
import pandas as pd
import numpy as np

import re

from transformers import AutoTokenizer, AutoModel

from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist

import torch

from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity

from scipy.spatial.distance import pdist, squareform

# import nltk
# from nltk.corpus import stopwords

# import spacy

from tqdm import tqdm

In [4]:
tqdm.pandas()

#EDA


In [5]:
# Connecting Google Drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
folder_path = "/content/drive/My Drive/hackathon/"

## Исследуем файл dealer

In [7]:
dealer = pd.read_csv(folder_path + "marketing_dealer.csv", sep=';', index_col='id')

In [8]:
display(dealer)

,name
id,
1,Moi_vibor_WB
2,Akson
3,Bafus
5,Castorama
6,Cubatora
7,Komus
9,Megastroy
10,OnlineTrade
11,Petrovich


In [9]:
dealer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 1 to 17
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    18 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


## Исследуем файл product

In [10]:
product = pd.read_csv(folder_path + "marketing_product.csv", sep=';', index_col='id')

In [11]:
display(product.sample(5))

,Unnamed: 0,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
id,,,,,,,,,,,,,,
44,24,283-08,4.680008e+12,"Средство для мытья полов и стен Multipower ""По...",77.94,182.00,NaN,NaN,"Средство для мытья полов и стен Multipower ""По...",NaN,NaN,NaN,NaN,NaN
121,486,124-1,4.680008e+12,Средство для удаления гипсовой пылиDuty Whiteк...,148.00,347.00,40.0,Средство для удаления гипсовой пыли PROSEPT Du...,Средство для удаления гипсовой пыли PROSEPT Du...,Средство для удаления гипсовой пыли PROSEPT Du...,189522870.0,149705847.0,124-1,NaN
446,88,0024-06 б12,NaN,Герметик акриловый цвет белый ф/п 600 мл. (12 ...,3686.40,7716.96,25.0,NaN,Герметик акриловый цвет белый ф/п 600 мл. (12 ...,NaN,NaN,NaN,NaN,NaN
67,323,112-0,4.680008e+12,Средство для удаления плесени с дезинфицирующ...,90.96,212.00,53.0,"Спрей для удаления плесени PROSEPT Bath Fungi,...","Спрей для удаления плесени PROSEPT Bath Fungi,...","Спрей для удаления плесени PROSEPT Bath Fungi,...",412005574.0,149705840.0,112-0,112-00
220,5,051-6,4.610093e+12,Грунт БЕТОНКОНТАКТготовый состав / 6 кг,703.00,1339.00,26.0,"Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...","Грунт бетоноконтакт, для гладких поверхностей ...",453193200.0,149699633.0,051-6,NaN


In [12]:
# Удаляем лишние столбцы


columns_to_drop = ["Unnamed: 0", "article", "ean_13", "cost", "category_id", "ozon_article", "wb_article",
                   "ym_article", "wb_article_td", "ozon_name", "name_1c", "wb_name"]

# Удаление столбцов из датафрейма
product = product.drop(columns=columns_to_drop)

In [13]:
product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 245 to 52
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               494 non-null    object 
 1   recommended_price  491 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.6+ KB


## Исследуем файл key

In [14]:
key = pd.read_csv(folder_path + "marketing_productdealerkey.csv", sep=';', index_col='id')

In [15]:
display(key.sample(5))

,key,dealer_id,product_id
id,,,
1650,15881530,15,167
1265,26390949,16,357
1109,3972364,13,142
1667,40019175,1,397
2014,530308963,18,416


In [16]:
key.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1700 entries, 1 to 2023
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key         1700 non-null   object
 1   dealer_id   1700 non-null   int64 
 2   product_id  1700 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 53.1+ KB


## Исследуем файл price

In [17]:
price = pd.read_csv(folder_path + "marketing_dealerprice.csv", sep=';', parse_dates=['date'], index_col='id')

In [18]:
display(price.sample(5))

,product_key,price,product_url,product_name,date,dealer_id
id,,,,,,
8279,15881618,164.0,https://www.vseinstrumenti.ru/product/kontsent...,Концентрат для мытья полов и стен PROSEPT Mult...,2023-07-17,15
2650,1425732,172.0,https://www.onlinetrade.ru/catalogue/sredstva_...,Чистящий спрей PROSEPT COOKY Spray для столешн...,2023-07-12,10
4635,3732187,189.0,https://www.sima-land.ru/4292830,"""Нейтрализатор запаха Flox Sea I """"Морской бри...",2023-07-13,13
10324,200544129,872.0,https://www.bafus.ru/200544129/,Просепт Professional Bath Uni средство для душ...,2023-07-19,3
1855,44586174,499.0,https://www.wildberries.ru/catalog/44586174/de...,Активная пена Universal Spray усиленное чистящ...,2023-07-11,1


In [19]:
price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20416 entries, 2 to 20570
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   product_key   20416 non-null  object        
 1   price         20416 non-null  float64       
 2   product_url   20182 non-null  object        
 3   product_name  20416 non-null  object        
 4   date          20416 non-null  datetime64[ns]
 5   dealer_id     20416 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.1+ MB


#Объединяем данные


In [20]:
df = price.merge(dealer, left_on='dealer_id', right_on='id')
df = df.merge(key, left_on='product_key', right_on='key')
df = df.merge(product, left_on='product_id', right_on='id')

In [21]:
df.head(5)

,product_key,price,product_url,product_name,date,dealer_id_x,name_x,key,dealer_id_y,product_id,name_y,recommended_price
0,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
1,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-12,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
3,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-13,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
4,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-14,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17567 entries, 0 to 17566
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   product_key        17567 non-null  object        
 1   price              17567 non-null  float64       
 2   product_url        17355 non-null  object        
 3   product_name       17567 non-null  object        
 4   date               17567 non-null  datetime64[ns]
 5   dealer_id_x        17567 non-null  int64         
 6   name_x             17567 non-null  object        
 7   key                17567 non-null  object        
 8   dealer_id_y        17567 non-null  int64         
 9   product_id         17567 non-null  int64         
 10  name_y             17567 non-null  object        
 11  recommended_price  17567 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 1.7+ MB


### Удаляем лишние данные.

In [23]:
# Список столбцов, которые нужно удалить
columns_to_drop = ["product_key", "product_url", "date", "name_x", "key", "dealer_id_x"]

# Удаление столбцов из датафрейма
df = df.drop(columns=columns_to_drop)

In [24]:
# Переименовываем столбец
df = df.rename(columns={'name_y':'original_name'})

# Выявляем типичные отличия в наименовании товара

In [25]:
# Установка максимальной ширины столбцов для отображения полного текста
pd.set_option('display.max_columns', None)  # Отображение всех столбцов
pd.set_option('display.max_colwidth', None)  # Отображение полного текста в ячейках
df.sample(20)

,price,product_name,dealer_id_y,product_id,original_name,recommended_price
14485,1452.0,Просепт Optic Crystal+ средство для мытья стекол и зеркал концентрат (5 л),3,203,Средство для мытья стекол и зеркалOptic C+концентрат 1:50-1:100 / 5 л,1452.0
13613,729.0,"Жидкое средство для стирки PROSEPT CRYSTAL для стирки светлых тканей, 3 л",10,157,Жидкое моющее средство для стирки белых и светлых тканейCrystal Whiteконцентрат / 3 л,1070.0
3625,139.0,"Чистящее средство PROSEPT Bath Acid Plus для удаления ржавчины и минеральных отложений, концентрат, 750 мл",10,52,"Средство усиленного действия для удаления ржавчины и минеральных отложений Bath Acid + концентрат 1:200-1:500 / 0,75 л",176.0
11296,3343.0,Отбеливатель для древесины Prosept 50 концентрат 1:1 20л,16,243,Отбеливатель для древесиныPROSEPT 50концентрат 1:1 / 20 л,5127.0
12258,952.0,Строительный антисептик PROSEPT,18,289,Антисептик против жуков и других биопоражении АНТИЖУК / 10 л,1129.0
13346,307.0,Средство для мытья посуды Prosept Cooky 1 л,12,84,"Гель для мытья посуды вручную. Без запахаCookyконцентрированное средство / 0,5 л",174.0
1988,229.0,Cпрей для очистки полков в банях и саунах Prosept Universal Wood 500мл,2,20,"Спрей для очистки полков в банях и саунах Universal Woodготовый состав / 0,5л",218.0
11083,274.0,Просепт Flox Чарующий Стамбул нейтрализатор неприятных запахов (400 мл),3,196,"Нейтрализатор неприятных запаховFlox «Чарующий Стамбул» готовый состав / 0,4 л",274.0
17473,8634.0,"Огнебиозащита для древесины PROSEPT I группа, красный, готовый состав, 65 кг 063-65 И",15,315,"ОГНЕБИОЗАЩИТА I группа, красный готовый состав / 65 кг",9868.0
4749,487.0,Жидкое средство для стирки черных тканей Crystal ко,1,158,Жидкое моющее средство для стирки белых и светлых тканейCrystal Blackконцентрат / 1 л,436.0


# Выявляем типичные отличия в наименовании товара

In [26]:
# Список столбцов, которые нужно удалить
columns_to_drop = ["price", "dealer_id_y", "recommended_price"]

# Удаление столбцов из датафрейма
df = df.drop(columns=columns_to_drop)

In [27]:
extra_words = (
    r'(?:готовый\sсостав|для|концентрат|просепт|prosept|средство|невымываемый|гелеобразный|канистра|'
    r'чистящее|спрей|универсальный|универсальная|универсальное|пэт|жидкое|моющее|гель|чистки|'
    r'концентрированное|professional|готовое|superrubber)'
)

def clean_description(text):
    text = text.lower()
    text = re.sub(r'\(\d+\)', '', text)
    text = re.sub(r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])', ' ', text)
    text = re.sub(r'\b\d+:\d+\b', '', text)
    text = re.sub(r'\b\d+(?::\d+)?[-:]\d+(?::\d+)?\b', '', text)

    # Замена выражений типа "2 группа" на "2-ая группа"
    text = re.sub(r'(\d+)\sгруппа', r'\1-ая группа', text)

    # Замена объема в литрах и веса в килограммах на миллилитры и граммы
    text = re.sub(r'(\d+[,.]\d+)\s*л', lambda x: f"{int(float(x.group(1).replace(',', '.')) * 1000)} мл", text)
    text = re.sub(r'(\d+[,.]\d+)\s*кг', lambda x: f"{int(float(x.group(1).replace(',', '.')) * 1000)} г", text)

    # Добавление пробелов между цифрами и буквами
    text = re.sub(r'(?<=\d)(?=[а-яА-Яa-zA-Z])', ' ', text)
    text = re.sub(r'(?<=[а-яА-Яa-zA-Z])(?=\d)', ' ', text)

    # Удаление выражений вида "007-10бц"
    text = re.sub(r'\b\d+-\d+[а-яА-Яa-zA-Z]*\b', '', text)

    # Удаление слов, соединенных через "/" или точку
    text = re.sub(r'\b\w+[/\.]\w+(?:[/\.]\w+)?\b', '', text)

    # Удаление избыточных слов
    text = re.sub(extra_words, '', text)

    # Удаление запятых, кавычек, дефисов и других символов
    text = re.sub(r'[,"“”/\.()–;+]', '', text)

    # Удаление обособленных пробелами дефисов
    text = re.sub(r'\s-\s', ' ', text)

    # Замена отдельных выражений
    text = re.sub(r'литров|литра|литр', 'л', text)
    text = re.sub(r'\b(мыло-гель|гель-мыло)\b', 'мыло', text)
    text = re.sub(r'osb\s+proplast', 'osbproplast', text)
    text = re.sub(r'\bкор\b', 'коричневый', text)
    text = re.sub(r'eco\s+ultra', 'ecoultra', text)
    text = re.sub(r'гриля\s+и\s+духовых\s+шкафов|удаления\s+пригоревших\s+жиров', 'жироудалитель', text)
    text = re.sub(r'грунт\s+пропиточный', 'грунтовка', text)
    text = re.sub(r'удалитель\s+высолов', 'очиститель фасадов', text)
    text = re.sub(r'с\s+itrus', 'citrus', text)

    return text

# Применяем функцию к столбцам product_name и original_name
df['product_name_normalized'] = df['product_name'].apply(clean_description)
df['original_name_normalized'] = df['original_name'].apply(clean_description)

In [28]:
df.sample(20, random_state=42)

,product_name,product_id,original_name,product_name_normalized,original_name_normalized
14843,Спрей антимикробное д/душ.кабин/акриловых ванн PROSEPT Bath Uni 500мл концентрат,65,"Универсальный концентрат для санитарных комнат с антимикробным эффектомBath Uni готовый состав / 0,5 л",антимикробное акриловых ванн bath uni 500 мл,санитарных комнат с антимикробным эффектом bath uni 500 мл
15011,Средство для удаления цемента Prosept Cement Cleaner 1 л,401,Удалитель цемента CEMENT CLEANER концентрат 1:2 / 1 л,удаления цемента cement cleaner 1 л,удалитель цемента cement cleaner 1 л
6991,"Жидкое средство для стирки PROSEPT CRYSTAL для стирки спортивной одежды, 3 л",153,"Жидкое моющее средство для стирки спортивной одежды, обуви и пуховиковCrystal Sportконцентрат / 3 л",стирки crystal стирки спортивной одежды 3 л,стирки спортивной одежды обуви и пуховиков crystal sport 3 л
1769,"Шпатлевка для плит OSB PROSEPT Proplast 1,3 кг 081-1",234,"Шпатлевка для плит OSBProplast / 1,3 кг",шпатлевка плит osbproplast 1300 г,шпатлевка плит osbproplast 1300 г
1085,АнтисептикProseptECOULTRAкор5л,275,Антисептик ECO ULTRA коричневый / 5 л,антисептик ecoultra коричневый 5 л,антисептик ecoultra коричневый 5 л
16322,"Антисептик PROSEPT АНТИПЛЕСЕНЬ, 5 литров, против грибка и плесени (025-5)",290,Антисептик против грибка и плесени АНТИПЛЕСЕНЬ готовый состав / 5 л,антисептик антиплесень 5 л против грибка и плесени,антисептик против грибка и плесени антиплесень 5 л
3115,"Жидкое мыло PROSEPT Diona Сitrus Е цитрусовый аромат, 5л, ПЭТ",178,жидкое гель-мыло эконом-класса. C ароматом яблокаDiona Citrus Eготовое средство / 5 л ПЭТ,мыло diona citrus е цитрусовый аромат 5 л,-мыло эконом-класса c ароматом яблока diona citrus e 5 л
3379,Просепт Professional Cooky Grill Gel концентрат гелеобразный для чистки гриля и духовых шкафов (500 мл),74,"Средство для чистки гриля и духовых шкафовCooky Grill Gelконцентрированное средство 1:1 – 1:50 / 0,5 л",cooky grill gel жироудалитель 500 мл,жироудалитель cooky grill gel 500 мл
8256,Просепт Bio Lasur антисептик лессирующий защитно-декоративный (2.7 л),332,"Антисептик лессирующий BiO LASUR / белый люкс / 2,7 л",bio lasur антисептик лессирующий защитно-декоративный 2700 мл,антисептик лессирующий bio lasur белый люкс 2700 мл
2830,"Жироудалитель Prosept 0,5 л спрей",72,"Средство для чистки гриля и духовых шкафовCooky Grill концентрированное средство / 0,55 л",жироудалитель 500 мл,жироудалитель cooky grill 550 мл


In [29]:
df.sample(20)

,product_name,product_id,original_name,product_name_normalized,original_name_normalized
13280,Просепт Professional Cooky White средство для мытья и отбеливания посуды (5 л),107,Средство для мытья и отбеливания посуды с дезинфицирующим эффектом.Cooky Whiteконцентрат 1:50-1:100 / 5 л,cooky white мытья и отбеливания посуды 5 л,мытья и отбеливания посуды с дезинфицирующим white 5 л
16747,"Кондиционер для белья PROSEPT Crystal Rinser ""Лепестки Сакуры"" концентрат 2 л",145,Кондиционер для белья с ароматом лепестков СакурыCrystal Rinserконцентрат / 2 л,кондиционер белья crystal rinser лепестки сакуры 2 л,кондиционер белья с ароматом лепестков сакуры crystal rinser 2 л
6592,"Средство для удаления граффити, маркера, краски PROSEPT Duty Graffiti аэрозоль 0.4 л 153-04",130,"Средство для удаления граффити, маркера, краскиDuty Graffiti готовый состав, аэрозоль / 0,4 л",удаления граффити маркера краски duty graffiti аэрозоль 400 мл,удаления граффити маркера краски duty graffiti аэрозоль 400 мл
14572,"Гель д/удаления ржавчины, минеральных отложений PROSEPT Bath Extra 0,75л концентрат",56,"Гелеобразное средство усиленного действия для удаления ржавчины и минеральных отложенийBath Extraконцентрат 1:10-1:100 / 0,75 л",ржавчины минеральных отложений bath extra 750 мл,гелеобразное усиленного действия удаления ржавчины и минеральных отложений bath extra 750 мл
14112,"Гель для чистки гриля и духовых шкафов Cooky Grill Gel. Концентрат, 5л",76,Средство для чистки гриля и духовых шкафовCooky Grill Gelконцентрированное средство 1:1 – 1:50 / 5 л,жироудалитель cooky grill gel 5 л,жироудалитель cooky grill gel 5 л
11115,Просепт Professional Flox Prof нейтрализатор неприятных запахов (500 мл),193,"Нейтрализатор неприятных запаховFLOX PROFготовый состав / 0,5 л",flox prof нейтрализатор неприятных запахов 500 мл,нейтрализатор неприятных запахов flox prof 500 мл
17225,Противогололедное средство PROSEPT Антилед-32 12 кг 242-12,3,Антигололед - 32 PROSEPTготовый состав / 12 кг,противогололедное антилед-32 12 кг,антигололед 32 12 кг
6774,"Средство эконом-класса для мытья полов Multipower E, концентрат, 5л",31,Концентрат эконом-класса для мытья половMultipower Eконцентрат 1:5-1:150 / 5 л,эконом-класса мытья полов multipower e 5 л,эконом-класса мытья полов multipower e 5 л
14359,"Антисептик PROSEPT FUNGI STOP Защита от плесени, концентрат, 1.0 л",394,Защита от плесени FUNGI STOP концентрат 1:10 / 1 л,антисептик fungi stop защита от плесени 1000 мл,защита от плесени fungi stop 1 л
8587,Антисептик лессирующий декоративный Prosept Biolasur белый люкс 0.9л,331,"Антисептик лессирующий BiO LASUR / белый люкс / 0,9 л",антисептик лессирующий декоративный biolasur белый люкс 900 мл,антисептик лессирующий bio lasur белый люкс 900 мл


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17567 entries, 0 to 17566
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_name              17567 non-null  object
 1   product_id                17567 non-null  int64 
 2   original_name             17567 non-null  object
 3   product_name_normalized   17567 non-null  object
 4   original_name_normalized  17567 non-null  object
dtypes: int64(1), object(4)
memory usage: 823.5+ KB


In [31]:
# Подсчет количества дубликатов
num_duplicates = df.duplicated().sum()

print(f"Количество дубликатов в датафрейме: {num_duplicates}")

Количество дубликатов в датафрейме: 15877


In [32]:
# Удаление полных дубликатов
df = df.drop_duplicates()

## Создаем эмбеддинги наименований при помощи двуязычной версии BERT.

In [33]:
# Загрузка мультиязычного токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

In [34]:
# Функция для подсчета длин токенов
def get_token_lengths(column):
    token_lengths = [len(tokenizer.tokenize(text)) for text in df[column]]
    return np.mean(token_lengths), np.median(token_lengths), np.max(token_lengths)

# Подсчитываем длину токенизированных текстов для каждого столбца
avg_length_product, median_length_product, max_length_product = get_token_lengths('product_name_normalized')
avg_length_original, median_length_original, max_length_original = get_token_lengths('original_name_normalized')

# Вывод статистики
print(f"Для 'product_name_normalized': Средняя длина: {avg_length_product}, Медианная длина: {median_length_product}, Максимальная длина: {max_length_product}")
print(f"Для 'original_name_normalized': Средняя длина: {avg_length_original}, Медианная длина: {median_length_original}, Максимальная длина: {max_length_original}")

Для 'product_name_normalized': Средняя длина: 15.228402366863905, Медианная длина: 15.0, Максимальная длина: 36
Для 'original_name_normalized': Средняя длина: 15.662130177514793, Медианная длина: 15.0, Максимальная длина: 32


In [35]:
# Функция для генерации эмбеддингов
def generate_embeddings(text):
    encoded_input = tokenizer(text, return_tensors='pt', max_length=36, padding='max_length',
                              truncation=True, add_special_tokens=True, return_attention_mask=True)
    # Выключение расчета градиентов для ускорения процесса
    with torch.no_grad():
        # Получение выходных данных модели
        output = model(**encoded_input)
    # Возвращение усредненных эмбеддингов последнего слоя
    return output.last_hidden_state.mean(dim=1).squeeze().numpy()

In [36]:
%%time

# Генерация эмбеддингов для каждого столбца с индикатором прогресса
df['product_name_embeddings'] = df['product_name_normalized'].progress_apply(generate_embeddings)
df['original_name_embeddings'] = df['original_name_normalized'].progress_apply(generate_embeddings)

100%|██████████| 1690/1690 [02:21<00:00, 11.92it/s]

CPU times: user 18min 53s, sys: 2.65 s, total: 18min 56s
Wall time: 4min 44s


In [37]:
# Проверка результатов
print(df[['product_name_embeddings', 'original_name_embeddings']].head())

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1690 entries, 0 to 17564
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_name              1690 non-null   object
 1   product_id                1690 non-null   int64 
 2   original_name             1690 non-null   object
 3   product_name_normalized   1690 non-null   object
 4   original_name_normalized  1690 non-null   object
 5   product_name_embeddings   1690 non-null   object
 6   original_name_embeddings  1690 non-null   object
dtypes: int64(1), object(6)
memory usage: 105.6+ KB


## Создание модели ранжирования и расчет ее точности.

Поскольку нам важно, чтобы в предложенных вариантах (желательно на первом месте) оказался верный вариант, то метрикой качества должна быть recall.

In [55]:
# Функция для расчета Recall at K
def recall_at_k(predicted, correct, k=5):
    return int(correct in predicted[:k])

In [58]:
%%time
# Собираем все уникальные эмбеддинги товаров с сайта заказчика
unique_original_embeddings = df.drop_duplicates(subset=['product_id'])[['product_id', 'original_name_embeddings']]
unique_embeddings_matrix = np.stack(unique_original_embeddings['original_name_embeddings'].values)

# Ранжирование товаров и подсчет Recall at 5
total_recall = 0
for index, row in df.iterrows():
    # Расчет расстояний
    distances = pdist(np.vstack([row['product_name_embeddings'], unique_embeddings_matrix]), 'cosine')
    distance_matrix = squareform(distances)[0, 1:]
    top_5_indices = np.argsort(distance_matrix)[:5]  # Топ-5 индексов с наименьшими расстояниями

    # Получаем ID топ-5 товаров
    top_5_products_ids = unique_original_embeddings.iloc[top_5_indices]['product_id'].values

    # Добавляем recall для текущего товара
    total_recall += recall_at_k(top_5_products_ids, row['product_id'])

average_recall = total_recall / len(df)
print(f"Средний Recall на Топ-5: {average_recall}")

Средний Recall на Топ-5: 0.8201183431952662
CPU times: user 1min 26s, sys: 63.2 ms, total: 1min 26s
Wall time: 1min 26s


## Лемматизация

In [44]:
# %%time

# # Downloading stop words and list of names
# nltk.download('punkt', quiet=True)
# nltk.download('stopwords', quiet=True)

In [45]:
# # Загрузка модели для русского языка
# spacy_model = "ru_core_news_sm"
# nlp = spacy.load(spacy_model)

In [46]:
# %%time

# def lem_text(txt):

#     # Processing the text with SpaCy
#     doc = nlp(txt)

#     # Filtering out stop words and names, and lemmatizing the remaining words
#     txt = [token.lemma_ for token in doc if not token.is_stop]

#     # Joining the words back into a string
#     txt = ' '.join(txt)

#     # Removing spaces at the beginning of the string
#     txt = txt.lstrip()

#     return txt

# # Применяем нормализацию к столбцам product_name и original_name
# df['product_name_lemmatized'] = df['product_name_normalized'].apply(lem_text)
# df['original_name_lemmatized'] = df['original_name_normalized'].apply(lem_text)

In [47]:
# df.sample(20, random_state=42)